# Prebuilt ReAct Agent - Dynamic Models & Prompt Customization

## Define tools explicitly

In [ ]:
import requests
import yfinance as yf
from pprint import pformat
from langchain_core.tools import tool

@tool("lookup_stock")
def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


@tool("fetch_stock_data")
def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}

## Define the basic agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from IPython.display import Image, display

agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o-mini"),
    tools=[lookup_stock_symbol, fetch_stock_data_raw],
    prompt="You are a helpful assistant. Answer clearly and simply."
)

display(Image(agent.get_graph().draw_mermaid_png()))

## Dynamic Model Selector

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.runtime import Runtime
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver


tools = [lookup_stock_symbol, fetch_stock_data_raw]


def dynamic_model_selector(state: AgentState, runtime: Runtime):
    """Select model depending on the query intent."""
    model = None

    user_msg = state["messages"][-1].content.lower()
    if any(word in user_msg for word in ["analyze", "invest", "risks"]):
        print("Select Model for Heavy reasoning → more powerful model (gpt-4)")
        model=ChatOpenAI(model="gpt-4")
    elif "summarize" in user_msg:
        print("Select Model for Quick summarization → lightweight model (gpt-4o-mini)")
        model=ChatOpenAI(model="gpt-4o-mini")
    else:
        print("Select Model for Default fallback (gpt-3.5-turbo)")
        model=ChatOpenAI(model="gpt-3.5-turbo")
    
    return model.bind_tools(tools)


agent = create_react_agent(
    model=dynamic_model_selector,
    tools=tools,
    prompt="You are a helpful assistant. Answer clearly and simply.",
    checkpointer=InMemorySaver()
)

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

messages = agent.invoke({"messages": [HumanMessage(content="Hi!")]}, config)
for message in messages['messages']:
    message.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage

messages = agent.invoke({"messages": [HumanMessage(content="summarize what HITL is")]}, config)
for message in messages['messages']:
    message.pretty_print()

## Dynamic Prompt Modifier

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from IPython.display import Image, display
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import InMemorySaver


def prompt_modifier(state):
    """Change the agent's behavior depending on the query intent."""
    user_msg = state["messages"][-1].content.lower()

    if "invest" in user_msg or "risks" in user_msg:
        prompt="You are a financial advisor. Give clear analysis of risks and opportunities."
    elif "summarize" in user_msg:
        prompt="You are a summarizer. Keep the answer short and clear."
    elif "explain" in user_msg:
        prompt="You are a teacher. Explain concepts step by step in simple terms."
    else:
        prompt="You are a helpful assistant. Answer clearly and simply."
    
    print(f"Selected prompt: {prompt}")

    # Filter out any existing system messages
    non_system_messages = [msg for msg in state["messages"] if msg.type != "system"]

    # Return the new system message + all non-system messages
    return [SystemMessage(content=prompt)] + non_system_messages


agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o-mini"),
    tools=[lookup_stock_symbol, fetch_stock_data_raw],
    prompt=prompt_modifier,
    checkpointer=InMemorySaver()
)

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

messages = agent.invoke({"messages": [HumanMessage(content="Hi!")]}, config)
for message in messages['messages']:
    message.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage

messages = agent.invoke({"messages": [HumanMessage(content="summarize what HITL is")]}, config)
for message in messages['messages']:
    message.pretty_print()

## Combining All Together

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.runtime import Runtime
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import SystemMessage, HumanMessage


tools = [lookup_stock_symbol, fetch_stock_data_raw]


def dynamic_model_selector(state: AgentState, runtime: Runtime):
    """Select model depending on the query intent."""
    model = None

    user_msg = state["messages"][-1].content.lower()
    if any(word in user_msg for word in ["analyze", "invest", "risks"]):
        print("Select Model for Heavy reasoning → more powerful model (gpt-4-turbo)")
        model=ChatOpenAI(model="gpt-4-turbo")
    elif "summarize" in user_msg:
        print("Select Model for Quick summarization → lightweight model (gpt-4o-mini)")
        model=ChatOpenAI(model="gpt-4o-mini")
    else:
        print("Select Model for Default fallback (gpt-3.5-turbo)")
        model=ChatOpenAI(model="gpt-3.5-turbo")
    
    return model.bind_tools(tools)


def prompt_modifier(state):
    """Change the agent's behavior depending on the query intent."""
    user_msg = state["messages"][-1].content.lower()

    if "invest" in user_msg or "risks" in user_msg:
        prompt="You are a financial advisor. Give clear analysis of risks and opportunities."
    elif "summarize" in user_msg:
        prompt="You are a summarizer. Keep the answer short and clear."
    elif "explain" in user_msg:
        prompt="You are a teacher. Explain concepts step by step in simple terms."
    else:
        prompt="You are a helpful assistant. Answer clearly and simply."
    
    print(f"Selected prompt: {prompt}")

    # Filter out any existing system messages
    non_system_messages = [msg for msg in state["messages"] if msg.type != "system"]

    # Return the new system message + all non-system messages
    return [SystemMessage(content=prompt)] + non_system_messages


agent = create_react_agent(
    model=dynamic_model_selector,
    tools=tools,
    prompt=prompt_modifier,
    checkpointer=InMemorySaver()
)

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "4"}}

messages = agent.invoke({"messages": [HumanMessage(content="What’s Tesla’s stock symbol?")]}, config)
for message in messages['messages']:
    message.pretty_print()

From the debug messages you can see that both `dynamic_model_selector` and `prompt_modifier` were triggered twice — once per node. This means you can adjust the model and prompt separately at each node, rather than only once for the whole graph.

In [ ]:
messages = agent.invoke({"messages": [HumanMessage(content="Summarize Tesla’s financials.")]}, config)
for message in messages['messages']:
    message.pretty_print()

## Reference Links

**LangGraph Prebuilt ReAct Agent (create_react_agent)**

https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent

→ API reference for the create_react_agent function. This function builds a ready-to-use ReAct agent graph from a language model, tool list, and optional configuration. It supports static or dynamic model selection, prompt customization, memory, hooks, human‑in‑the‑loop, and other advanced features.